Importing necessary libraries

In [310]:
from geopy.geocoders import Nominatim
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [311]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

retrieve standard table using BeautifulSoup

In [312]:
toronto_wiki = requests.get(wikipedia_link)

In [313]:
toronto_wiki = requests.get(wikipedia_link)
soup = BeautifulSoup(toronto_wiki.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Naming Columns

In [314]:
df.columns = df.loc[0,].astype(str)
df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [315]:
df.drop(0, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Grouping Values that are repeated 

In [316]:
df1 = df.groupby(['Postcode', 'Borough']).agg(lambda col: ', '.join(col))
df1.reset_index(inplace = True)
print(df1.shape)
df1.head()

(180, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


Deleating Not Assigned Values

In [337]:
df2 = df1[df1['Borough'] != 'Not assigned']
print(df2.shape)
df2.head()

(103, 3)


,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [338]:
df2 = df2.reset_index()
df2.head(10)

,index,Postcode,Borough,Neighbourhood
0,1,M1B,Scarborough,"Rouge, Malvern"
1,2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,4,M1G,Scarborough,Woburn
4,5,M1H,Scarborough,Cedarbrae
5,6,M1J,Scarborough,Scarborough Village
6,7,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,9,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,10,M1N,Scarborough,"Birch Cliff, Cliffside West"


Create empty latlong dataframe

In [322]:
# define the dataframe columns
column_names = ['Latitude', 'Longitude'] 

# instantiate the dataframe
LatLong = pd.DataFrame(columns=column_names)

create function to add coordinate values

In [323]:
def get_lat_long(Neighbourhood, Borough):
    
    if (df2.loc[i, 'Neighbourhood'].find(',') == -1):
         Neighbourhood = Neighbourhood
    else:
         Neighbourhood = df2.loc[i, 'Neighbourhood'][0:df2.loc[i, 'Neighbourhood'].find(',')]
    address = '{} {} Toronto'.format(Neighbourhood, Borough)

    geolocator = Nominatim(user_agent="Coursera")
    location = geolocator.geocode(address)
    try:
        latitude = location.latitude
        longitude = location.longitude
    except:
        return 0, 0
    
    
    return latitude, longitude

Add Values to dataframe

In [324]:
LatLong = pd.DataFrame(columns=column_names)
for i in range(103):
    a = get_lat_long(df2.loc[i, 'Neighbourhood'], df2.loc[i, 'Borough'])
    time.sleep(0.1)
    LatLong.loc[i,'Latitude'] = a[0]
    LatLong.loc[i,'Longitude'] = a[1]
    

In [339]:
LatLong.head()

,Latitude,Longitude
0,43.8049,-79.1658
1,43.7901,-79.1733
2,43.7549,-79.1978
3,43.7598,-79.2253
4,43.7565,-79.2267


Concatenate

In [340]:
df = pd.concat([df2, LatLong], axis = 1)
df.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,1,M1B,Scarborough,"Rouge, Malvern",43.8049,-79.1658
1,2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7901,-79.1733
2,3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7549,-79.1978
3,4,M1G,Scarborough,Woburn,43.7598,-79.2253
4,5,M1H,Scarborough,Cedarbrae,43.7565,-79.2267
